In [1]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../src')

In [3]:
import ontime as on
import pandas as pd
import numpy as np

from ontime.module.processing.tensorflow.utils import create_dataset
from ontime.module.processing.common import train_test_split, normalize

# TensorFlow Dataset

## Create test TimeSeries

In [4]:
v1 = np.arange(0,100)
v2 = np.arange(100,200)
arr = np.column_stack((v1,v2))

start_date = pd.Timestamp('2023-01-01')
time_indices = pd.date_range(start=start_date, periods=len(arr), freq='D')

ts = on.TimeSeries.from_times_and_values(time_indices, arr)

## Create Splits

In [6]:
train, test = train_test_split(ts, test_split=0.3)
train_enc, transformer = normalize(train, return_transformer=True)
test_enc = transformer.transform(test)

## Create DataSets

In [7]:
ds_train = create_dataset(
    train_enc,
    window_length = 10,
    stride_length = 10,
    input_length = 6,      
    target_length = 4
)

ds_test = create_dataset(
    test_enc,
    window_length = 10,
    stride_length = 10,
    input_length = 6,      
    target_length = 4
)

I0000 00:00:1736180054.254876  319935 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


## Check samples encoding

Get a sample

In [8]:
# Input
input = None
target = None

for i in ds_train.take(1):
    input = i[0]
    target = i[1]

2025-01-06 17:14:44.385253: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Print input

In [9]:
transformer.inverse_transform(on.TimeSeries.from_data(input.numpy().T))

<TimeSeries (DataArray) (time: 6, component: 2, sample: 1)> Size: 96B
array([[[  0.],
        [100.]],

       [[  1.],
        [101.]],

       [[  2.],
        [102.]],

       [[  3.],
        [103.]],

       [[  4.],
        [104.]],

       [[  5.],
        [105.]]])
Coordinates:
  * time       (time) int64 48B 0 1 2 3 4 5
  * component  (component) <U1 8B '0' '1'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

Print target

In [10]:
transformer.inverse_transform(on.TimeSeries.from_data(target.numpy().T))

<TimeSeries (DataArray) (time: 4, component: 2, sample: 1)> Size: 64B
array([[[  6.],
        [106.]],

       [[  7.],
        [107.]],

       [[  8.],
        [108.]],

       [[  9.],
        [109.]]])
Coordinates:
  * time       (time) int64 32B 0 1 2 3
  * component  (component) <U1 8B '0' '1'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None